# Projekt 1 - SPI - podsumowanie

## Grupa: Klaudia Buksa, Grzegorz Cichy, Marcin Słyś



## 1. Pobieranie i łączenie danych

Pierwszym etapem wykonania projektu było pobranie danych dotyczących opadów deszczu ze strony *https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/opad/*. Zawiera ona informacje o opadach dla stacji w całym kraju od ru 1950 do 2024. Do realizacji zadania wykorzystane zostały dane od 1991 roku. 

**Informacje o pobieranych plikach csv:**
 - Pliki składają się z następujących kolumn:

    Kod stacji<br>
    Nazwa stacji<br>
    Rok<br>
    Miesiąc<br>
    Dzień<br>
    Suma dobowa opadów [mm]<br>
    Status pomiaru SMDB<br>
    Rodzaj opadu       [S/W/ ]<br>
    Wysokość pokrywy śnieżnej [cm]<br>
    Status pomiaru PKSN<br>
    Wysokość świeżospałego śniegu [cm]<br>
    Status pomiaru HSS<br>
    Gatunek śniegu  [kod]<br>
    Status pomiaru GATS<br>
    Rodzaj pokrywy śnieżnej [kod]<br>
    Status pomiaru RPSN<br>

- Kluczowe pod kątem obliczania wartości SPI są kolumny związane z czasem pomiaru ('Rok','Miesiąc','Dzień'), kolumna 'Suma dobowa opadów [mm]', na bazie której obliczany był współczynnik SPI oraz kolumny związane z miejscem pomiaru - 'Kod stacji' i 'Nazwa Stacji'.
- Wśród kolumn brakuje geometrii stacji, które pozwoliłyby na tworzenie przestrzennych wizualizacji.

<br>
W celu uzyskania geomterii pobrany został także plik kody_stacji.csv, który jest wykazem stacji i zawiera długość oraz szerokość geograficzną większości z nich.

<br><br>
Po pobraniu danych, kolejnym etapem jest stworzenie jednego data frame'u zawierającego dane dla wszystkich stacji leżących na terenie analizowanego województwa - Podkarpackiego. Ze względu na olbrzymią ilość dostępnych do analizy danych, najpierw obróbce poddany został plik z kodami stacji, który następnie był łączony z każdym wczytywanym plikiem. Dzięki temu do złączonego zestawu danych trafiały obserwacje jedynie dla podkarpackich stacji.
<br><br>
**Przetwarzanie pliku 'kody_stacji.csv':**

1. Wczytanie danych.
2. MOdyfikacja kolumn Długości i Szerokości geograficznej w formacie 'Stopnie Minuty Sekundy' na format dziesiętny.
3. Konwersja do GeoDataFrame przy użyciu obliczonych współrzędnych.
4. Ustawienie CRS na 'EPSG:9702'.
5. Wczytanie pliku shapefile z granicami województw.
6. Złączenie przestrzenne stacji z województwem.
7. Wybranie stacji, do których przypisane zostało województwo Podkarpackie.

<br>

**Mapa stacji na tle województwa**
<br>
!['Stacje'](../images/plot_stacje_mapa.png)

<br>
Najwięcej stacji znajduje się na południu województwa w górach, natomiast najmniej w jego północnej części - w okolicach Stalowej Woli.
<br><br>

**Merge plików csv z danymi pogodowymi:**

1. Wyszukanie plików o rozszerzeniu .csv w odpowiednim katalogu.
2. Stworzenie pustego wyjściowego DataFrame'u.
3. Wczytanie każdego znalezionego pliku i merge z plikiem zawierającym stacje, który usuwa wszystkie stacje spoza zadanego obszaru.
4. Konkatenacja danych z wyjściowym DataFrame. 
5. Zapis do pliku.

<br><br>
Po wykonaniu tych transformacji w katalogu data/raw zapisany został plik 'data_merged.csv', który mógł zostać poddany obróbce w kolejnych krokach.


## 2. Eksploracyjna analiza danych i preprocessing

Dane przygotowane po pobraniu zapisane w pliku "merged_data.csv" zostały wczytane i została dokonana wstępna eksploracyjna analiza danych.

W pierwszej kolejności sprawdzone zostały wartości brakujące - w kilku kolumnach przekraczały one wartości kilkudziesięciu procent. Ze względu na duże ilości brakujących wartości oraz niewielki wpływ w kontekście dalszych analiz, usunięte zostają kolumny dotyczące statusu pomiaru:
- Status pomiaru SMDB,
- Status pomiaru PKSN,
- Status pomiaru HSS,     
- Status pomiaru GATS,      
- Status pomiaru RPSN.

Oprócz nich, usunięte zostały kolumny mające niewielki wpływ w kontekście dalszych analiz:
- Gatunek śniegu[kod]
- Rodzaj pokrywy śnieżnej [kod]
- Rzeka

Jedyną kolumną posiadającą wartości brakujące był "Rodzaj opadu [S/W/ ]", gdzie pusta wartość była oznaką braku opadu. W kontekście jakości danych została ona uzupełniona wartością "brak", tak aby dane były pełne. Kolumna ta została również zakodowana, tak by przechowywać wartości True/False, aby dane były lepiej dostosowane pod modele data science.

W kontekście projektu SPI, jedyną istotną kolumną jest "Suma dobowa opadów [mm]", więc resztę kolumn można by również usunąć, jednak taki zbiór może być przydatny w kontekście innych zagadnień, w związku z tym został tak przygotowany, by być gotowym do użycia w innych projektach (np. modelach predykcji).

W kolejnym kroku skupiono się już jedynie na interesującej kolumnie, jaką była "Suma dobowa opadów [mm]". Przygotowany został histogram, aby zobaczyć rozkład tej zmiennej.

!['Histogram'](../images/histogram.png)

Aby zmienić dystrybucję rozkładu zmiennej "Suma dobowa opadów [mm]" zostanie wykonana transformacja Box-Cox. Aby uniknąć problemu z obsługą danych z wartością 0, została dodana do nich wartość 1e-6, która nie przekłamuje danych, gdyż jej znaczenie jest niewielkie, a pozwala obsłużyć wszystkie wartości.

!['Histogram Box-Cox'](../images/box-cox_histogram.png)

Tak przygotowane dane były gotowe do przeprowadzenia obliczeń i analiz SPI

## 3. Obliczanie SPI (Standarized Precipitation Index)

SPI obliczane były dla jednej stacji w trzech wariantach:
- SPI-1: suma opadów z jednego danego miesiąca,
- SPI-3: suma opadów z danego miesiąca i dwóch miesięcy przed,
- SPI-12: suma opadów z danego miesiąca i 11 miesięc przed.

Dla każdego z obliczeń została przygotowana funkcja, która składała się z następujących kroków:
- pogrupowanie danych dla każdej stacji i miesiąca i obliczenie średniej,
- wybranie danych z sumą dobową opadów większą od 0,
- w przypadku SPI-3 i SPI-12 zsumowanie odpowiedniej ilości miesięcy,
- obliczenie parametrów rozkładu gamma,
- obliczenie prawdopodobieństwa dla danych opadów za pomocą rozkładu gamma,
- zapisanie wartości w ramce danych,
- zapis ramki danych do pliku csv.

Pomimo przekształceń wykonanych w poprzednim kroku, nie została użyta tutaj przekształcona kolumna, gdyż algorytm źle sobie radził z wartościami poniżej 0, które pojawiły się po transformacji. Użycie parametrów rozkładu gamma pozwoliło w tym przypadku otrzymać dobre jakościowo dane.

## 4. Analiza

W procesie analizy dane dla każdego wariantu SPI, zostały złączone z danymi zawierającymi geometrię stacji i odpowiednio sklasyfikowane w zależnie od wartości SPI:
- SPI 2.0 lub więcej: ekstremalnie mokro
- SPI od 1.5 do 1.99: bardzo mokro
- SPI od 1.0 do 1.49: umiarkowanie mokro
- SPI od -0.99 do 0.99: średnie warunki
- SPI od -1.49 do -1.0: umiarkowana susza
- SPI od -1.99 do -1.5: silna susza
- SPI -2.0 lub mniej: ekstremalna susza

Wykonane zostały wykresy SPI dla każdego z punktów pomiarowych, tak aby zaobserwować ich zmianę na przestrzeni lat. Poniżej zaprezentowane są wykresy dla wszystkich trzech wariantów SPI stacji w Wetlinie - niewielkiej wsi w Bieszczadach będącej częstym punktem startu wędrówek po górskich szlakach.

!['Wetlina SPI1'](../images/plot_WETLINA.png)

!['Wetlina SPI3'](../images/plot_spi3_WETLINA.png)

!['Wetlina SPI12'](../images/plot_spi12_WETLINA.png)

W tym przypadku z wykresu zmian SPI-1, ciężko określić jakiś trend wskaźnika - pokazuje to, że lata były różne - raz bardziej wilgotne, raz bardziej suche.

W przypadku SPI-3 dużo bardziej widoczne są zmiany - widać wyraźne wzrosty, a także utrzymujące się spadki w poszczególnych przedziałach czasowych.

Na wykresie SPI-12 dość dokładnie widoczne są okresy lat suchych i wilgotnych. Szczególnie mokry okres to końcówka lat 90. XX wieku, po ktorym nastąpiło pewne unormowanie i potem kolejne wzrosty w okolicy roku 2002 i na przełomie 2004/2005.

!['Wetlina SPI_wszystkie'](../images/plot_spi_all_WETLINA.png)

Na wykresie z wszystkimi wariantami można zauważyć, że mniej więcej do przełomu lat 1996/1997 wskaźnik był w normie, po czym zaczęły się lata bardziej mokre. Nie odnotowano zbyt wielu ekstremalnie suchych periodów - na wykresie SPI-1 widzimy jedynie trzy takie przypadki.

Warto jednak zauważyć, że skala czasowa ma wyraźny wpływ na obserwowane zmiany - agregując dane z ostatnich 12 miesięcy dużo lepiej wygląda ogólny trend wskaźnika.

Następnie sporządzone zostały wykresy kołowe wszystkich wariantów SPI, aby zobaczyć jak się rozkładają dla wszystkich regionów.

!['SPI1 kołowy'](../images/spi1_kolowy.png)

!['SPI3 kołowy'](../images/spi3_kolowy.png)

!['SPI12 kołowy'](../images/spi12_kolowy.png)

Dla każdego z wariantów przeważają wartości średnie. Wszystkie wartości prezentują się dość symetrycznie - ilość okresów suchych i mokrych, a także ich wersji silnych i ekstremalnych jest porównywalna, przy czym im wyższe wartości SPI, ich udział jest zdecydowanie mniejszy. Można więc wywnioskować że w większości stacji warunki są stabilne z przeplatającymi się periodami suchymi, czy też mokrymi.